In [1]:
import numpy as np
from sklearn.datasets import load_iris

In [2]:
# Construct a list that contains the number of neurons of each layer in your network.
structure = input("Enter the number of neurons of each layer in your network").split(' ')
structure = [int(x) for x in structure]
print(structure)    

[4, 2, 2, 3]


In [9]:
# Construct a list that contains all the bias vectors of your network(_n means it has all vectors)
np.random.seed(123)
B_n = [np.random.randn(l, 1) for l in structure[1:]]
print("bias vector of each layer\n")
for B in B_n:
    print(B)
    print()

bias vector of each layer

[[-1.0856306 ]
 [ 0.99734545]]

[[ 0.2829785 ]
 [-1.50629471]]

[[-0.57860025]
 [ 1.65143654]
 [-2.42667924]]



In [10]:
# Construct a list that contains all the weight matrices of your network(_n means it has all matrices)
np.random.seed(123)
W_n = [np.random.randn(l, next_l) for l, next_l in zip(structure[:-1], structure[1:])]
print("weight matrix of each layer\n")
for W in W_n:
    print(W)
    print()

weight matrix of each layer

[[-1.0856306   0.99734545]
 [ 0.2829785  -1.50629471]
 [-0.57860025  1.65143654]
 [-2.42667924 -0.42891263]]

[[ 1.26593626 -0.8667404 ]
 [-0.67888615 -0.09470897]]

[[ 1.49138963 -0.638902   -0.44398196]
 [-0.43435128  2.20593008  2.18678609]]



##### Don't forget W_n & B_n is of list where a set of numpy.array are contained.

##### Load training and test datasets

In [11]:
load_iris().keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [67]:
data = load_iris()['data']
target = load_iris()['target']

print(type(data), type(target))
print(data.shape, target.shape)
print(len(data), len(target))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(150, 4) (150,)
150 150


In [68]:
# Just pick up 5 indexes out of dataset randomly
indexes = [0, 52, 108, 49, 142, 88]

data = data[indexes]
target = target[indexes]

print(data)
print(target)

[[5.1 3.5 1.4 0.2]
 [6.9 3.1 4.9 1.5]
 [6.7 2.5 5.8 1.8]
 [5.  3.3 1.4 0.2]
 [5.8 2.7 5.1 1.9]
 [5.6 3.  4.1 1.3]]
[0 1 2 0 2 1]


In [71]:
X = data
print(X)
print()

Y = np.zeros((6, 3))
for i in range(6):
    for j in range(3):
        Y[i, target[i]] = 1
print(Y)

[[5.1 3.5 1.4 0.2]
 [6.9 3.1 4.9 1.5]
 [6.7 2.5 5.8 1.8]
 [5.  3.3 1.4 0.2]
 [5.8 2.7 5.1 1.9]
 [5.6 3.  4.1 1.3]]

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]


In [73]:
# Provide sigmoid and sigmoid_derivative function
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

def sigmoid_derivative(x):
    return sigmoid(x) * (1.0 - sigmoid(x))

In [ ]:
# Create two lists that contain pre and post activation vector of each layer, respectively
Z_n, A_n = [], []

# Forward pass layer by layer from L=0(First hidden layer) thru L=H(Output layer)
for i, (b, W) in enumerate(zip(B_n, W_n)):
    if i == 0:
        z = np.dot(np.array(W).T, X).reshape(-1, 1) + np.array(b)
    else:
        z = np.dot(np.array(W).T, a).reshape(-1, 1) + np.array(b)
    a = sigmoid(z)

    Z_n.append(z)
    A_n.append(a)

In [75]:
# Create two lists that contain pre and post activation vector of each layer, respectively
Z_n, A_n = [], []
H = len(structure) - 2

for x, y in zip(X,Y):
    # Forward pass layer by layer from L=0(First hidden layer) thru L=H(Output layer)
    for i, (b, W) in enumerate(zip(B_n, W_n)):
        if i == 0:
            z = np.dot(np.array(W).T, x).reshape(-1, 1) + np.array(b)
        else:
            z = np.dot(np.array(W).T, a).reshape(-1, 1) + np.array(b)
        a = sigmoid(z)

        Z_n.append(z)
        A_n.append(a)
        
    # Initialize a list called e_Je_W_ns that will contain e_Je_W matrices for each layer
    e_Je_W_ns = [np.zeros(W.shape) for W in W_n]    # (4, 2), (2, 2), (2, 3)

    # Initialize a list called e_Je_B_ns that will contain e_Je_B vectors for each layer
    e_Je_B_ns = [np.zeros(B.shape) for B in B_n]    # (2, 1), (2, 1), (3, 1)
    
    for L in range(H, -1, -1):
        if L != H:
            delta = sigmoid_derivative(Z_n[L]) * np.dot(W_n[L+1], delta)
        else:
            delta = sigmoid_derivative(Z_n[L]) * (A_n[L] - y.reshape(-1, 1))
        e_Je_B_ns[L] = delta
        # print(f"{L} : {delta}")
        
        if L != 0:
            e_Je_W_ns[L] = np.dot(A_n[L-1], delta.T)
        else:
            e_Je_W_ns[L] = np.dot(X.reshape(-1, 1), delta.T)

In [76]:
for x in e_Je_W_ns:
    print(x)
    print()

[[ 3.32640856e-04 -6.69606436e-03]
 [ 2.28282940e-04 -4.59533829e-03]
 [ 9.13131761e-05 -1.83813532e-03]
 [ 1.30447394e-05 -2.62590759e-04]
 [ 4.50043511e-04 -9.05938120e-03]
 [ 2.02193461e-04 -4.07015677e-03]
 [ 3.19596116e-04 -6.43347361e-03]
 [ 9.78355458e-05 -1.96943070e-03]
 [ 4.36998771e-04 -8.79679044e-03]
 [ 1.63059243e-04 -3.28238449e-03]
 [ 3.78297444e-04 -7.61513202e-03]
 [ 1.17402655e-04 -2.36331683e-03]
 [ 3.26118486e-04 -6.56476899e-03]
 [ 2.15238201e-04 -4.33274753e-03]
 [ 9.13131761e-05 -1.83813532e-03]
 [ 1.30447394e-05 -2.62590759e-04]
 [ 3.78297444e-04 -7.61513202e-03]
 [ 1.76103982e-04 -3.54497525e-03]
 [ 3.32640856e-04 -6.69606436e-03]
 [ 1.23925025e-04 -2.49461221e-03]
 [ 3.65252704e-04 -7.35254126e-03]
 [ 1.95671092e-04 -3.93886139e-03]
 [ 2.67417158e-04 -5.38311057e-03]
 [ 8.47908063e-05 -1.70683994e-03]]

[[ 4.45316977e-05 -1.02837818e-05]
 [ 4.33761983e-02 -1.00169403e-02]]

[[ 0.04947257 -0.00744208  0.00341264]
 [ 0.02044892 -0.0030761   0.00141058]]



##### Forward Pass

In [72]:
# Provide sigmoid and sigmoid_derivative function
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

def sigmoid_derivative(x):
    return sigmoid(x) * (1.0 - sigmoid(x))

In [26]:
np.dot(W_n[0].T, X).reshape(-1,1) + B_n[0]

array([[-11.84317054],
       [  8.27689919]])

In [27]:
# Create two lists that contain pre and post activation vector of each layer, respectively
Z_n, A_n = [], []
H = len(structure) - 2

# Forward pass layer by layer from L=0(First hidden layer) thru L=H(Output layer)
for i, (b, W) in enumerate(zip(B_n, W_n)):
    if i == 0:
        z = np.dot(np.array(W).T, X).reshape(-1, 1) + np.array(b)
    else:
        z = np.dot(np.array(W).T, a).reshape(-1, 1) + np.array(b)
    a = sigmoid(z)

    Z_n.append(z)
    A_n.append(a)
    
for L in range(H, -1, -1):
    if L != H:
        delta = sigmoid_derivative(Z_n[L]) * np.dot(W_n[L+1], delta)
    else:
        delta = sigmoid_derivative(Z_n[L]) * (A_n[L] - y)
    e_Je_B_ns[L] = delta
    # print(f"{L} : {delta}")
    
    if L != 0:
        e_Je_W_ns[L] = np.dot(A_n[L-1], delta.T)
    else:
        e_Je_W_ns[L] = np.dot(X.reshape(-1, 1), delta.T)

In [28]:
for z, a in zip(Z_n, A_n):
    print(np.round(z, 4))
    print(np.round(a, 4))
    print()

[[-11.8432]
 [  8.2769]]
[[0.    ]
 [0.9997]]

[[-0.3957]
 [-1.601 ]]
[[0.4023]
 [0.1678]]

[[-0.0515]
 [ 1.7646]
 [-2.2383]]
[[0.4871]
 [0.8538]
 [0.0964]]



##### Backpropagation 

In [31]:
# Initialize a list called e_Je_W_ns that will contain e_Je_W matrices for each layer in the network.
e_Je_W_ns = [np.zeros(W.shape) for W in W_n]    # (4, 2), (2, 2), (2, 3)

# Initialize a list called e_Je_B_ns that will contain e_Je_B vectors for each layer in the network.
e_Je_B_ns = [np.zeros(B.shape) for B in B_n]    # (2, 1), (2, 1), (3, 1)

In [36]:
y, A_n[2]

(array([[0],
        [1],
        [0]]),
 array([[0.48713827],
        [0.85378893],
        [0.0963659 ]]))

In [43]:
np.dot(W_n[2], sigmoid_derivative(Z_n[2]) * (A_n[2] - y))

array([[ 0.18944364],
       [-0.07477456]])

In [47]:
H = len(structure) - 2

for L in range(H, -1, -1):
    if L != H:
        delta = sigmoid_derivative(Z_n[L]) * np.dot(W_n[L+1], delta)
    else:
        delta = sigmoid_derivative(Z_n[L]) * (A_n[L] - y)
    e_Je_B_ns[L] = delta
    # print(f"{L} : {delta}")
    
    if L != 0:
        e_Je_W_ns[L] = np.dot(A_n[L-1], delta.T)
    else:
        e_Je_W_ns[L] = np.dot(X.reshape(-1, 1), delta.T)

##### Implement Gradient Descent

In [54]:
learning_rate = 0.1

for W, e_Je_W, B, e_Je_B in zip(W_n, e_Je_W_ns, B_n, e_Je_B_ns):
    W -= learning_rate * e_Je_W
    B -= learning_rate * e_Je_B

In [53]:
for W, e_Je_W in zip(W_n, e_Je_W_ns):
    print(f"W의 shape : {W.shape},    e_Je_W의 shape : {e_Je_W.shape}")
    
print()

for B, e_Je_B in zip(B_n, e_Je_B_ns):
    print(f"B의 shape : {B.shape},     e_Je_B의 shape : {e_Je_B.shape}")

W의 shape : (4, 2),    e_Je_W의 shape : (4, 2)
W의 shape : (2, 2),    e_Je_W의 shape : (2, 2)
W의 shape : (2, 3),    e_Je_W의 shape : (2, 3)

B의 shape : (2, 1),     e_Je_B의 shape : (2, 1)
B의 shape : (2, 1),     e_Je_B의 shape : (2, 1)
B의 shape : (3, 1),     e_Je_B의 shape : (3, 1)


In [48]:
for x in e_Je_W_ns:
    print(x)
    print()

[[ 2.68546237e-06 -4.26149872e-05]
 [ 1.43864056e-06 -2.28294574e-05]
 [ 1.96614210e-06 -3.12002585e-05]
 [ 6.23410908e-07 -9.89276488e-06]]

[[ 3.27416513e-07 -7.50649946e-08]
 [ 4.55425009e-02 -1.04412803e-02]]

[[ 0.04896634 -0.00734351  0.00337623]
 [ 0.02042727 -0.00306349  0.00140846]]



In [49]:
for x in e_Je_B_ns:
    print(x)
    print()

[[ 4.79546853e-07]
 [-7.60981914e-06]]

[[ 0.04555408]
 [-0.01044394]]

[[ 0.12170398]
 [-0.01825202]
 [ 0.0083915 ]]

